# 实验参数设置

In [12]:
from FederatedLearning.learner import CNNModel

clients_num = 10 # 客户端总数
rotation_cycle = 5 # 角色轮换间隔轮数
max_round = 10 # 联邦学习轮数
model_class = CNNModel # 选用CNN模型

## 数据切分

In [13]:
import data_split
# 使用data_split模块中的函数获取PyTorch数据集
train_dataset = data_split.get_mnist_pytorch_dataset(train=True)
test_dataset = data_split.get_mnist_pytorch_dataset(train=False)
# test
# 使用正确的数据集对象创建数据加载器
client_dataloaders = data_split.create_client_dataloaders(train_dataset, clients_num // 2, 64) * 2
client_test_loader = data_split.create_client_dataloaders(test_dataset,clients_num // 2, 64) * 2

2025-10-17 10:18:33,566 - DGS_BCFL - INFO - 
为5个客户端分配数据集...
2025-10-17 10:18:33,568 - DGS_BCFL - INFO - 客户端 1 数据准备完成，样本数: 12000
2025-10-17 10:18:33,569 - DGS_BCFL - INFO - 客户端 2 数据准备完成，样本数: 12000
2025-10-17 10:18:33,570 - DGS_BCFL - INFO - 客户端 3 数据准备完成，样本数: 12000
2025-10-17 10:18:33,572 - DGS_BCFL - INFO - 客户端 4 数据准备完成，样本数: 12000
2025-10-17 10:18:33,573 - DGS_BCFL - INFO - 客户端 5 数据准备完成，样本数: 12000
2025-10-17 10:18:33,578 - DGS_BCFL - INFO - 
为5个客户端分配数据集...
2025-10-17 10:18:33,579 - DGS_BCFL - INFO - 客户端 1 数据准备完成，样本数: 2000
2025-10-17 10:18:33,581 - DGS_BCFL - INFO - 客户端 2 数据准备完成，样本数: 2000
2025-10-17 10:18:33,583 - DGS_BCFL - INFO - 客户端 3 数据准备完成，样本数: 2000
2025-10-17 10:18:33,584 - DGS_BCFL - INFO - 客户端 4 数据准备完成，样本数: 2000
2025-10-17 10:18:33,585 - DGS_BCFL - INFO - 客户端 5 数据准备完成，样本数: 2000


## 开始训练

In [14]:
from owner import Owner
from client import Client

# 初始化管理者，并获得初始化字典
owner = Owner(rotation_cycle=1, model_class=model_class)
main_dict = owner.get_main_dict()

# 初始化客户端以及身份
clients = [owner]
for i in range(clients_num):
    client_name = f"client_{i+1}"
    client = Client(epochs=2, client_name=client_name, data_loader=client_dataloaders[i], test_loader=client_test_loader[i], ModelClass=model_class, main_dict=main_dict)
    owner.join(client_name)
    clients.append(client)


import threading
for _ in range(max_round):
    t = [threading.Thread(target=client.run) for client in clients]
    _ = [i.start() for i in t]
    _ = [i.join() for i in t]

2025-10-17 10:18:33,888 - DGS_BCFL - INFO - [client_1] 当前轮次角色: aggregator
2025-10-17 10:18:33,889 - DGS_BCFL - INFO - [client_2] 当前轮次角色: validator
2025-10-17 10:18:33,889 - DGS_BCFL - INFO - [client_3] 当前轮次角色: validator
2025-10-17 10:18:33,890 - DGS_BCFL - INFO - [client_4] 当前轮次角色: validator
2025-10-17 10:18:33,891 - DGS_BCFL - INFO - [client_5] 当前轮次角色: learner
2025-10-17 10:18:33,892 - DGS_BCFL - INFO - [client_6] 当前轮次角色: learner
2025-10-17 10:18:33,894 - DGS_BCFL - INFO - [client_7] 当前轮次角色: learner
2025-10-17 10:18:33,895 - DGS_BCFL - INFO - [client_8] 当前轮次角色: learner
2025-10-17 10:18:33,896 - DGS_BCFL - INFO - [client_9] 当前轮次角色: learner
2025-10-17 10:18:33,898 - DGS_BCFL - INFO - [client_10] 当前轮次角色: learner
2025-10-17 10:18:33,961 - DGS_BCFL - INFO - Aggregator使用设备: cuda
2025-10-17 10:18:33,983 - DGS_BCFL - INFO - [client_4] 验证者初始化...
2025-10-17 10:18:33,992 - DGS_BCFL - INFO - [client_2] 验证者初始化...
2025-10-17 10:18:34,000 - DGS_BCFL - INFO - [client_3] 验证者初始化...
2025-10-17 10:18:34,

KeyboardInterrupt: 

## 计算结果可视化